In [3]:
import numpy as np
from tqdm.notebook import tqdm

import torch
import torchvision
import torchvision.transforms as transforms

In [4]:
# MNIST, CIFAR10, CIFAR100
trainset = torchvision.datasets.MNIST(root='./datasets/', train=True,
                                        download=True, transform=transforms.ToTensor())

testset = torchvision.datasets.MNIST(root='./datasets/', train=False,
                                       download=True, transform=transforms.ToTensor())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./datasets/MNIST\raw\train-images-idx3-ubyte.gz to ./datasets/MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./datasets/MNIST\raw\train-labels-idx1-ubyte.gz to ./datasets/MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./datasets/MNIST\raw\t10k-images-idx3-ubyte.gz to ./datasets/MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./datasets/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./datasets/MNIST\raw



In [5]:
print(trainset)

classes = trainset.classes
print(len(classes))

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./datasets/
    Split: Train
    StandardTransform
Transform: ToTensor()
10


In [6]:
print(testset)

classes = testset.classes
print(len(classes))

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./datasets/
    Split: Test
    StandardTransform
Transform: ToTensor()
10


In [7]:
print(trainset[0])
print(trainset[0][0].shape)

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000

In [8]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=60000,
                                          shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=True)

In [9]:
dataiter = iter(trainloader)
tr_images, tr_labels = dataiter.next()

In [10]:
tr_images, tr_labels = tr_images.numpy(), tr_labels.numpy()
print(tr_images.shape)
print(tr_labels.shape)

(60000, 1, 28, 28)
(60000,)


In [11]:
dataiter = iter(testloader)
te_images, te_labels = dataiter.next()

In [12]:
te_images, te_labels = te_images.numpy(), te_labels.numpy()
print(te_images.shape)
print(te_labels.shape)

(100, 1, 28, 28)
(100,)


In [29]:
before_sum=abs(tr_images-te_images[0])**2
before_sum.shape

(60000, 1, 28, 28)

In [46]:
distance=before_sum.sum(axis=3).sum(axis=1).sum(axis=1)
distance

array([108.88251 ,  81.69998 ,  90.19571 , ...,  65.587204,  84.77617 ,
       101.926865], dtype=float32)

In [55]:
top3_indices=np.argsort(distance)[:3]
top3_indices

array([48861,  8605, 25894], dtype=int64)

In [60]:
labels=tr_labels[top3_indices]
labels

array([1, 1, 1], dtype=int64)

In [67]:
from collections import Counter

cnt=Counter(labels)

cnt.most_common()[0][0]

1

In [94]:
from collections import Counter

class kNN():
    def __init__(self):
        print("kNN initialized..")
    
    #### HERE ####
    def fit(self, X, Y):
        self.num_data = len(X)
        self.Xtr = X
        self.Ytr = Y
    
    #### HERE ####
    def predict(self, X , k, p):
        
        num_test_data = len(X)
          
        # Initialize output matrix to zeros
        Ypred = np.zeros(num_test_data, dtype=self.Ytr.dtype) 
                
        # Loop over input test images matrix X
        for i in tqdm(range(num_test_data)):
            # calculate distance
            before_sum=abs(self.Xtr-X[i])**p
            distances = before_sum.sum(axis=3).sum(axis=1).sum(axis=1)
            distances=distances**(1/p)
            
            # choose k nearest neighbors
            topK = np.argsort(distance)[:k]
            
            # labels of kNNs
            votes = self.Ytr[topK]

            # returns the most common vote in votes list
            cnt=Counter(votes)
            res = cnt.most_common()[0][0]
            
            # Predicted y is the most commont vote
            Ypred[i] = res

        return Ypred

In [95]:
model = kNN()
model.fit(tr_images,tr_labels)

kNN initialized..


In [107]:
y_est = model.predict(te_images, k=10, p=2)

  0%|          | 0/100 [00:00<?, ?it/s]

In [108]:
y_est

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [109]:
sum(te_labels==y_est)

19

In [106]:
accuracy=sum(te_labels==y_est)/y_est.shape[0]*100
print(accuracy)

19.0
